## Understanding Uncertainty in Model Forecasts of Tropical Cyclones


Authors:
* Lara Tobias-Tarsh (laratt@umich.edu)
* Nick Dewhirst
* Ryan Pohlman
* Claire Sheeren

In [7]:
# perform imports
import numpy as np
import sys

# ensure that the path is always set to the correct location for the track error module
sys.path.insert(0, '/Users/laratobias-tarsh/Documents/clim323-final/errorCalcModules/')

import track_error as te # the name you use to import this is arbitrary, I just used te bc it is an easy shorthand
from pprint import pprint
import os

## Motivation
Tropical Cyclones are some of the largest and most destructive natural phenomena to occur on the planet. These warm core storms form in the tropics (30-0 degrees N & S), driven by the relatively warm sea surface temperatures adn more dominant vertical motions in dynamics. (expand)

### Numerical Weather Prediction (NWP)

### Project Goals



### Data
#### TC Best Tracks
Best Track data is gathered from the NHC HurDat2 database (Landsea et. al, 200x), which conatins storm tracks for all Atlantic Basin TCs from 1951 to present, and is updated at the end of each season. Data is stored in a .txt file, contaning all necessary data for calculating model errors. Because the dataset is very large and some datapoints are encoded, we use the hurdat2parser.py library to save time writing a custom parser. The data quality is good, with no missing values in the data for the 2020 Atlantic Hurricane Season. Because the 2020 season was so active that greek letter storm names had to be used alongside the regular alphabetical namelist, we did encounter a limitation of this library, as it uses only the first letter of the storm name to extract data for a given storm. This was resolved by encoding the unique storm identifier codes into our parser function. 

#### Model Forecasts

### DATA CLASSES

#### Reading in data
Data for this study was stored in custom data classes, allowing us to perform calculations efficiently on as many storms as we wanted without requiring us to repeat various lines of code or define vast amount of functions in the main notebook that would clutter our analysis. This allows us to initialise all necessary data for a TC from just the filepath to a folder containing the csv files for the track. Given that some longer lived TCs have a large number of TIGGE track data files, this approach streamlines reading in data to a simple class constructor.

For example, here we load the data for TC Laura. We call the initialiser function from the trackError module, and pass it the filepath to the TIGGE data. This function iterates through each file in the directory and calls another function which is built to extract and parse data for a specific storm. Documentation for these helper functions can be accessed either on the github at https://github.com/larantt/hurricane-verification or by calling the help() function on any of the helper functions:

In [5]:
help(te.generate_cyclone)

Help on function generate_cyclone in module track_error:

generate_cyclone(dirpath)
    Function initialises a cyclone object from a list of csvs
    
    Iterates through a list of csv files in a directory and constructs a
    cyclone object. Generates variables by parsing the name of the file.
    Requires following modified TropCy naming conventions. 
    Each TC's csv files should be contained in its own directory:
    e.g. ~/cyclones/Isaac-2006/
    
    Parameters
    ----------
        dirpath : str
            path to directory containing cyclone csvs
    
    Returns
    -------
         : Cyclone
            Cyclone object for storm in directory



As we can see from the docstring, this function will return the data as a 'Cyclone object.' This is the 'superclass' that contains all of the data for a given storm, although it is built on a number of other classes that help maintain format standardisation between storms.

In [6]:
help(te.Cyclone)

Help on class Cyclone in module track_error:

class Cyclone(builtins.object)
 |  Cyclone(name: str, year: int, ecmwf: track_error.Model, best_track: track_error.Track) -> None
 |  
 |  Main TC object containing all information wrt a TC
 |  
 |  Class contains all information that is used in tracking TCs for
 |  organisational purposes, as well as key TC information that can
 |  be used as a metric for comparison.
 |  
 |  Attributes
 |  ----------
 |      name : str 
 |          NHC assigned storm name
 |      year : int
 |          Year of storm occurrence
 |      ecmwf : Model 
 |          ECMWF model object
 |      gfs : Model 
 |          GFS model object
 |      number : int 
 |          Number of storm in season
 |      formation_date : dt.datetime
 |          Date of formation in best track
 |      best_track : List
 |          List containing best track data
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, name: str

The base class of a Cyclone is a Position, which contains the raw data for a TC at a given timestep from either the TIGGE or HurDat2 database. When initialising a Cyclone, we essentially loop through all data for a TC, connect the  